In [1]:
import pandas as pd


results_df = pd.DataFrame(
    {
        'Model': [],
        'Accuracy': [],
        'Recall': [],
        'ROC-AUC': [],
        'PR-AUC': [],
    }
).astype(
    {
        'Model': str,
        'Accuracy': float,
        'Recall': float,
        'ROC-AUC': float,
        'PR-AUC': float,
    }
)

In [2]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    CAT_FEATURES
)

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTENC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


pipeline_knn = ImbPipeline([
    ('smote', SMOTENC(categorical_features=CAT_FEATURES, random_state=42)),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid_knn = {
    'smote__k_neighbors': [3, 5, 7],
    'smote__sampling_strategy': [0.5, 0.7, 1.0],
    'knn__n_neighbors': [3, 5, 7, 11, 15, 21, 31, 51],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=param_grid_knn,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1,
)

grid_knn.fit(X, y)

print('KNN best params:', grid_knn.best_params_)
print('KNN best recall:', grid_knn.best_score_)

results_df = evaluate_and_append(
    model_name='KNeighborsClassifier+SMOTENC',
    best_estimator=grid_knn.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
KNN best params: {'knn__metric': 'manhattan', 'knn__n_neighbors': 51, 'knn__weights': 'uniform', 'smote__k_neighbors': 5, 'smote__sampling_strategy': 1.0}
KNN best recall: 0.6046941678520625
                          Model  Accuracy    Recall   ROC-AUC    PR-AUC
0  KNeighborsClassifier+SMOTENC  0.767685  0.604278  0.781483  0.113037


In [3]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    CAT_FEATURES
)

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

pipeline_dt = ImbPipeline([
    ('smote', SMOTENC(categorical_features=CAT_FEATURES, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
])

param_grid_dt = {
    'smote__sampling_strategy': [0.5, 0.7, 1.0],
    'smote__k_neighbors': [3, 5, 7],
    'dt__max_depth': [6, 8, 10, 12, 15],
    'dt__min_samples_leaf': [1, 2, 5, 10],
    'dt__min_samples_split': [2, 5, 10],
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_features': ['sqrt', 'log2']
}

grid_dt = GridSearchCV(
    estimator=pipeline_dt,
    param_grid=param_grid_dt,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

grid_dt.fit(X, y)

print('DecisionTree best params:', grid_dt.best_params_)
print('DecisionTree recall:', grid_dt.best_score_)

results_df = evaluate_and_append(
    model_name='DecisionTreeClassifier+SMOTENC',
    best_estimator=grid_dt.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits
DecisionTree best params: {'dt__criterion': 'entropy', 'dt__max_depth': 6, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2, 'smote__k_neighbors': 3, 'smote__sampling_strategy': 1.0}
DecisionTree recall: 0.6311522048364153
                            Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier+SMOTENC  0.767685  0.604278  0.781483  0.113037
1  DecisionTreeClassifier+SMOTENC  0.732446  0.631016  0.761697  0.100007


In [4]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    CAT_FEATURES
)

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


pipeline_rf = ImbPipeline([
    ('smote', SMOTENC(categorical_features=CAT_FEATURES, random_state=42)),
    ('rf', RandomForestClassifier(random_state=42, n_jobs=-1))
])

param_grid_rf = {
    'smote__sampling_strategy': [0.5, 0.7, 1.0],
    'smote__k_neighbors': [3, 5, 7],
    'rf__n_estimators': [300, 500],
    'rf__max_depth': [8, 12, None],
    'rf__min_samples_leaf': [2, 5],
    'rf__min_samples_split': [2, 5, 10],
    'rf__max_features': ['sqrt', 'log2'],
    'rf__criterion': ['gini', 'entropy']
}

grid_rf = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=param_grid_rf,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X, y)

print('RandomForest best params:', grid_rf.best_params_)
print('RandomForest best recall:', grid_rf.best_score_)

results_df = evaluate_and_append(
    model_name='RandomForestClassifier+SMOTENC',
    best_estimator=grid_rf.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
RandomForest best params: {'rf__criterion': 'entropy', 'rf__max_depth': 8, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 300, 'smote__k_neighbors': 3, 'smote__sampling_strategy': 1.0}
RandomForest best recall: 0.42802275960170705
                            Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier+SMOTENC  0.767685  0.604278  0.781483  0.113037
1  DecisionTreeClassifier+SMOTENC  0.732446  0.631016  0.761697  0.100007
2  RandomForestClassifier+SMOTENC  0.814409  0.427807  0.782174  0.112241
